# Time series clustering exercise

# Data

Download the census bureau business data for all years 1993-2014. You can investigate using the API (I have not done it with the census bureau). I did is as you see below

In [1]:
#these commands can be run on the shell and get the data with the command wget
#the cell needs to be run only once
#!for ((y=93; y<=99; y+=1)); do wget \
#ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

#!for ((y=0; y<=1; y+=1)); do wget \
#ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done

#!for ((y=2; y<=9; y+=1)); do wget \
#ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

#!for ((y=10; y<=14; y+=1)); do wget \
#ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done

 Download the NYC zipcodes shapefile. One of many ways in which you can get the zipcodes shapefile for NYC
 https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip


## TASKS:
    
    1. get and prep your data.
    2. cluster the NUMBER OF ESTABLISHMENTS time series with K-means in **a few** clusters (as discussed there is no real good, sound way to decide what a good number is here. try a few options, keeping in mind a few is more than a couple, but i recommand you stay within the single digit numbers)
    3. plot the cluster centers (if you used K means those are the means of the clusters). you can plot for example the cluster centers overlayed on each time series (using the alpha channel to control the opacity in the plot may be helpful here).
    4. Use another clustering algorithm (of your choice)
    5. overlay your data on a NYC map: you can use shapefiles for the zip codes and different colors for different clusters
    6. Compare the results of the 2 algorithms
    7. attempt an interpretation. this is dangerous ground: clustering is an exploratory tool so you do not want to jump to conclusions because you see some clusters! but seeing structure in your data can inform your next moves as an investigator. 
    

In [2]:
from __future__ import print_function, division
import sys
import os
import numpy as np
import pylab as pl
import pandas as pd
import geopandas as gpd
import fiona
import shapely
import json
import requests 
import urllib
import zipfile 

from scipy.cluster.vq import kmeans2, whiten
from sklearn import preprocessing
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import seaborn as sns
sns.set_style('whitegrid')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# 1. get and prep your data
## Read in NYC Zipcode shp as GeoDataFrame

In [3]:
PUIDATA = os.getenv('PUIDATA')
print(PUIDATA)

/home/cusp/vmr286/PUIdata


In [4]:
url = 'https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip'

urllib.request.urlretrieve(url, 'nyc.zip')

os.system("unzip -d %s puma.zip"%PUIDATA)


zipshp = gpd.GeoDataFrame.from_file(PUIDATA + '/ZIP_CODE_040114.shp')
zipshp.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,..."


## You can use zipfile module in python to unzip the files
it should be install in your system, but if it is not you can get the code with wget from here
https://github.com/python/cpython/blob/2.7/Lib/zipfile.py
remembering to use the raw link
(or you can use the usual shell commands, and miss the chance to learn something new)

In [5]:
zipshp['ZIPCODE'] = zipshp['ZIPCODE'].astype(int)
zipshp.dtypes

ZIPCODE         int64
BLDGZIP        object
PO_NAME        object
POPULATION    float64
AREA          float64
STATE          object
COUNTY         object
ST_FIPS        object
CTY_FIPS       object
URL            object
SHAPE_AREA    float64
SHAPE_LEN     float64
geometry       object
dtype: object

In [6]:
est = []

NYCzip = zipshp.ZIPCODE.unique()

for i in range(94,100):
    fname = 'zbp' + str(i) + 'totals.zip'
    zf = zipfile.ZipFile(fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    df.columns = [x.lower() for x in df.columns]
    df = (df[(df['zip'].astype(str).str.startswith('1'))])
    df = df[['zip','name','est']]
    df['year'] = i
    est.append(df)

for i in range(0,15):
    i = str(i).zfill(2)
    fname = 'zbp' + str(i) + 'totals.zip'
    zf = zipfile.ZipFile(fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    df.columns = [x.lower() for x in df.columns]
    df = (df[(df['zip'].astype(str).str.startswith('1'))])
    df = df[['zip','name','est']]
    df['year'] = i
    est.append(df)
    
est = pd.concat(est)

FileNotFoundError: [Errno 2] No such file or directory: 'zbp94totals.zip'

In [ ]:
est.shape

In [ ]:
est[::3000]

## Merge GeoDataFrame with DataFrame

In [ ]:
estshp = zipshp.merge(est, left_on='ZIPCODE', right_on='zip')
estshp.drop(['STATE','BLDGZIP','URL','zip','name'], inplace=True, axis=1)
estshp.head()

In [ ]:
type(estshp)

In [ ]:
estshp.shape

In [ ]:
est_ts = pd.pivot_table(estshp, values='est', index=['ZIPCODE'], columns='year')

est_ts.head()

In [ ]:
est_ts.shape

In [ ]:
years = list(est_ts.columns)
years

# Data cleaning

you may need to clean your data: for some NYC zip codes there may be no info
sanity check: you should have 20 (N_timestamps) datapoints per time series and about 250 zipcodes (Nzipcodes)


IMPORTANT: we talked about the importance of "whitening" your data.
Whitenings decorrelates the data: it makes the features independent so that the data covariance matrix is the identity matrix.
Whitening your data in time series analysis is in most cases **wrong**: you are modifying your time behaviour. This is because of the strong correlation between features (two consecutive time stamps for the same observation, the same zip code here, are strongly correlated). Here instead you want to standardize your time series: subtract the mean and divide each time series (separately) by its standard deviation. As a sanity check (if you use skitlearn Kmeans or skitlearns kmeans2): you want your data array to be shaped Nzipcodes x Ntimestamps

mydata.shape should be (Nzipcodes, Ntimestamps)

mydata[i].std() shoould be 1 for all i in range(len(Nzipcodes))

mydata[i].mean() should be ~0 for all i in range(len(Nzipcodes))



In [ ]:
est_ts = est_ts.dropna(0)

In [ ]:
NYCzip = est_ts.index
type(NYCzip)

In [ ]:
# standardize the time series
est_scaled = preprocessing.scale(est_ts)

# verifing the std of the standardized data is 1
est_scaled.std(axis=0)

In [ ]:
est_scaled.shape

In [ ]:
est_scaled = pd.DataFrame(est_scaled)
est_scaled.head()

In [ ]:
est_scaled.set_index(NYCzip, inplace=True)
est_scaled.columns = years

est_scaled.head()

# 2. cluster the NUMBER OF ESTABLISHMENTS time series with K-means in **a few** clusters

## define + run Elbow function to detect the number of clustering

In [ ]:
# This code is taken from Applied Data Sceince lab, NYU CUSP, Fall 2017

def elbow(data,K):
#data is your input as numpy form
#K is a list of number of clusters you would like to show.
    # Run the KMeans model and save all the results for each number of clusters
    KM = [KMeans(n_clusters=k).fit(data) for k in K]
    
    # Save the centroids for each model with a increasing k
    centroids = [k.cluster_centers_ for k in KM]

    # For each k, get the distance between the data with each center. 
    D_k = [cdist(data, cent, 'euclidean') for cent in centroids]
    
    # But we only need the distance to the nearest centroid since we only calculate dist(x,ci) for its own cluster.
    globals()['dist'] = [np.min(D,axis=1) for D in D_k]
    
    # Calculate the Average SSE.
    avgWithinSS = [sum(d)/data.shape[0] for d in dist]
    
    
    # elbow curve
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, avgWithinSS, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elbow for KMeans clustering')
    plt.show()
    
    
    # Total with-in sum of square plot. Another way to show the result.
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(data)**2)/data.shape[0]
    bss = tss-wcss
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, bss/tss*100, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Percentage of variance explained')
    plt.title('Elbow for KMeans clustering')
    plt.show()

In [ ]:
# This code is taken from Applied Data Sceince lab, NYU CUSP, Fall 2017

X=np.asarray(est_scaled.iloc[:,:-1])
range_n_clusters = [2, 3, 4, 5, 6]


for n_clusters in range_n_clusters:

    clusterer = KMeans(n_clusters=n_clusters, random_state=324)
    cluster_labels = clusterer.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)
    sample_silhouette_values = silhouette_samples(X, cluster_labels)
#elbow
elbow(X,range(1,15))

## Fig.1: elbow function to detect number of clusters.
According to the elbow, I will use _5 clasters_.

## Assigning each timeseries to a cluster

In [ ]:
clusters = KMeans(n_clusters=5, random_state=324)
cluster_labels = clusters.fit_predict(X)
est_scaled['cluster_labels'] = cluster_labels

est_scaled.head()

In [ ]:
cluster = est_scaled[['cluster_labels']]
cluster.head()

# 3. plot the cluster centers

In [ ]:
clustermean = est_scaled.groupby('cluster_labels').mean()
clustermean

In [ ]:
pl.figure(figsize=(12,8))
pl.plot(clustermean.ix[:,5:].T)

# 4. Use another clustering algorithm >>
# DBScan clustering

In [ ]:
import sklearn.cluster
from sklearn.cluster import DBSCAN


db = DBSCAN(eps=0.8, min_samples=2).fit(cluster)
labels = (db.labels_).astype(int)
num_clusters = len(set(labels)) - (1 if -1 in labels else 0) # the last statements removes outliers
clusters = pd.Series([cluster[labels == i] for i in range(num_clusters)])
print('Number of clusters: %d' % num_clusters)

In [ ]:
db = DBSCAN(eps=0.8, min_samples=2).fit(est_scaled)
db.labels_

In [ ]:
len(set(labels))

In [ ]:
#function to get the centroid from DBscan as mean of cluster members
# the cluster center is the mean of the cluster for both K Means and DBScan
def getCentroid(points):
    #print points[:,0], np.nanmean(points[:,0])
    return np.nanmean(points[:,0]), np.nanmean(points[:,1])

In [ ]:
# plot DBScan centroids
centroids = np.zeros((num_clusters,2))

for i in labels:
    centroids[i] = getCentroid(cluster[labels == i])
colorsc2 = get_colors(np.arange(num_clusters), pl.cm.jet)

pl.figure(figsize=(10, 6), dpi=100)
for i in np.unique(labels):
    if int(i) == -1:
        continue
    pl.scatter(centroids[i,0], centroids[i,1], c=colorsc2[i], alpha=.3, 
                s=sum(labels == i)*100, 
                label=["%i"%i])
#pl.legend(numpoints=1, scatterpoints=1)
colors2 = get_colors(labels, pl.cm.jet)

pl.scatter(cluster[:,0], cluster[:,1], c=colors2, s=20)
pl.scatter(cluster[:,0][labels==-1], cluster[:,1][labels==-1], c='k', alpha=0.4, s=300)
pl.xlabel("longitude (deg)", fontsize=18)
pl.ylabel("latitude(deg)", fontsize=18)
pl.title("DBscan clustering, %d culsters, %d outliers"%(num_clusters,
                                                        sum(labels == -1)), fontsize=18);

# 5. Overlay your data on a NYC map
## Merging w zipcode shp

In [ ]:
clustershp = zipshp.merge(cluster, left_on='ZIPCODE', right_index=True)
clustershp.drop(['PO_NAME','STATE','BLDGZIP','URL','ST_FIPS','CTY_FIPS'], inplace=True, axis=1)
clustershp.head()

In [ ]:
clustershp.dtypes

In [ ]:
clustershp.plot(column='cluster_labels', figsize=(10,10), cmap='Purples', edgecolor="black", lw=.2, legend=True, scheme='Fisher_Jenks')
pl.title('5 Clusters of buisness establishments by Zipcode, 1994-2014', fontsize=20)
pl.grid(False)


## Fig.2 Five Clusters of buisness establishments by Zipcode, 1994-2014
Clusters are observed in south Brooklyn and center SI. 

### The map of the clusters may look something like this

In [ ]:
...

### Figure 3: 
cloropleth of  cluster centers for 5 k-means clusters of business patterns (number of businesses) at the zipcode level for NYC zipcodes: each color indicates a cluster. The business pattern time series are plotted at the top.

### or maybe like this, depending on which algorithm you use, and how you proceed to preprocess your data and how you cluster it. There is no one correct answer, but general trends should be retrieved.

In [ ]:
...

### Figure 9: 
As figures 3, 5, 7 for hierarchical agglomerative clustering in 7 clusters, with smoothed time series

### And if you use hierarchical clustering and make a dandrogram it may look like this:

In [ ]:
from PIL import Image
Image.open("dandrogram.png")

Points: 4 - data not reproducible, all tasks not attempted.